# Test Model on VRPLib

In this notebook, we will test the trained model's performance on the VRPLib benchmark. We will use the trained model from the previous notebook.

[VRPLIB](http://vrp.galgos.inf.puc-rio.br/index.php/en/) is a collection of instances related to the CVRP, which is a classic optimization challenge in the field of logistics and transportation. 

## Before we start

To use the VRPLib, we strongly recomment to use the Python `vrplib` tool:

[VRPLib](https://github.com/leonlan/VRPLIB) is a Python package for working with Vehicle Routing Problem (VRP) instances. This tool can help us easily load the VRPLib instances and visualize the results.

## Installation

Uncomment the following line to install the package from PyPI. Remember to choose a GPU runtime for faster training!

> Note: You may need to restart the runtime in Colab after this


In [ ]:
# !pip install rl4co[graph] # include torch-geometric

## NOTE: to install latest version from Github (may be unstable) install from source instead:
# !pip install git+https://github.com/ai4co/rl4co.git

In [ ]:
# Install the `tsplib95` package
# !pip install vrplib

## Imports

In [8]:
%load_ext autoreload
%autoreload 2

import os
import re
import torch
import vrplib

from rl4co.envs import TSPEnv, CVRPEnv
from rl4co.models.zoo.am import AttentionModel
from rl4co.utils.trainer import RL4COTrainer
from rl4co.models.nn.utils import get_log_likelihood
from rl4co.models.zoo import EAS, EASLay, EASEmb, ActiveSearch

from tqdm import tqdm
from math import ceil
from einops import repeat

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load a trained model

In [15]:
# Load from checkpoint; alternatively, simply instantiate a new model
# Note the model is trained for CVRP problem
checkpoint_path = "../cvrp-20.ckpt" # modify the path to your checkpoint file

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load checkpoint
# checkpoint = torch.load(checkpoint_path)

lit_model = AttentionModel.load_from_checkpoint(checkpoint_path, load_baseline=False)
policy, env = lit_model.policy, lit_model.env
policy = policy.to(device)

/home/cbhua/miniconda3/envs/rl4co-user/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'env' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['env'])`.
/home/cbhua/miniconda3/envs/rl4co-user/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:198: Attribute 'policy' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['policy'])`.
/home/cbhua/miniconda3/envs/rl4co-user/lib/python3.10/site-packages/lightning/pytorch/core/saving.py:177: Found keys that are not in the model state dict but in the checkpoint: ['baseline.baseline.model.encoder.init_embedding.init_embed.weight', 'baseline.baseline.model.encoder.init_embedding.init_embed.bias', 'baseline.baseline.model.encoder.init_embedding.init_embed_depot.weight', 'baseline.baseline.model.encoder.init_

## Download vrp problems

In [11]:
problem_names = vrplib.list_names(low=50, high=200, vrp_type='cvrp') 

instances = [] # Collect Set A, B, E, F, M datasets
for name in problem_names:
    if 'A' in name:
        instances.append(name)
    elif 'B' in name:
        instances.append(name)
    elif 'E' in name:
        instances.append(name)
    elif 'F' in name:
        instances.append(name)
    elif 'M' in name and 'CMT' not in name:
        instances.append(name)

# Modify the path you want to save 
# Note: we don't have to create this folder in advance
path_to_save = './vrplib/' 

try:
    os.makedirs(path_to_save)
    for instance in tqdm(instances):
        vrplib.download_instance(instance, path_to_save)
        vrplib.download_solution(instance, path_to_save)
except: # already exist
    pass 

  0%|          | 0/37 [00:00<?, ?it/s]

100%|██████████| 37/37 [00:42<00:00,  1.16s/it]


In [12]:
# Utils function
def normalize_coord(coord:torch.Tensor) -> torch.Tensor:
    x, y = coord[:, 0], coord[:, 1]
    x_min, x_max = x.min(), x.max()
    y_min, y_max = y.min(), y.max()
    
    x_scaled = (x - x_min) / (x_max - x_min) 
    y_scaled = (y - y_min) / (y_max - y_min)
    coord_scaled = torch.stack([x_scaled, y_scaled], dim=1)
    return coord_scaled 

## Test the greedy

In [18]:
for instance in instances:
    problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))

    coords = torch.tensor(problem['node_coord']).float()
    coords_norm = normalize_coord(coords)
    demand = torch.tensor(problem['demand'][1:]).float()
    capacity = problem['capacity']
    n = coords.shape[0]

    # Prepare the tensordict
    batch_size = 2
    td = env.reset(batch_size=(batch_size,)).to(device)
    td['locs'] = repeat(coords_norm, 'n d -> b n d', b=batch_size, d=2)
    td['demand'] = repeat(demand, 'n -> b n', b=batch_size) / capacity
    td['visited'] = torch.zeros((batch_size, 1, n), dtype=torch.uint8)
    action_mask = torch.ones(batch_size, n, dtype=torch.bool)
    action_mask[:, 0] = False
    td['action_mask']  = action_mask

    # Get the solution from the policy
    with torch.no_grad():
        out = policy(td.clone(), decode_type='greedy', return_actions=True)

    # Calculate the cost on the original scale
    td['locs'] = repeat(coords, 'n d -> b n d', b=batch_size, d=2)
    neg_reward = env.get_reward(td, out['actions'])
    cost = ceil(-1 * neg_reward[0].item())

    # Load the optimal cost
    solution = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
    optimal_cost = solution['cost']

    # Calculate the gap and print
    gap = (cost - optimal_cost) / optimal_cost
    print(f'Problem: {instance:<15} Cost: {cost:<10} Optimal Cost: {optimal_cost:<10}\t Gap: {gap:.2%}')

Problem: A-n53-k7        Cost: 1371       Optimal Cost: 1010      	 Gap: 35.74%
Problem: A-n54-k7        Cost: 1426       Optimal Cost: 1167      	 Gap: 22.19%
Problem: A-n55-k9        Cost: 1333       Optimal Cost: 1073      	 Gap: 24.23%
Problem: A-n60-k9        Cost: 1728       Optimal Cost: 1354      	 Gap: 27.62%
Problem: A-n61-k9        Cost: 1297       Optimal Cost: 1034      	 Gap: 25.44%
Problem: A-n62-k8        Cost: 1818       Optimal Cost: 1288      	 Gap: 41.15%
Problem: A-n63-k9        Cost: 2166       Optimal Cost: 1616      	 Gap: 34.03%
Problem: A-n63-k10       Cost: 1698       Optimal Cost: 1314      	 Gap: 29.22%
Problem: A-n64-k9        Cost: 1805       Optimal Cost: 1401      	 Gap: 28.84%
Problem: A-n65-k9        Cost: 1592       Optimal Cost: 1174      	 Gap: 35.60%
Problem: A-n69-k9        Cost: 1641       Optimal Cost: 1159      	 Gap: 41.59%
Problem: A-n80-k10       Cost: 2230       Optimal Cost: 1763      	 Gap: 26.49%
Problem: B-n51-k7        Cost: 1270     

## Test the Augmentation

In [20]:
# Import augmented utils
from rl4co.data.transforms import (
    StateAugmentation as SymmetricStateAugmentation)
from rl4co.utils.ops import batchify, unbatchify

num_augment = 100
augmentation = SymmetricStateAugmentation(env.name, num_augment=num_augment)

for instance in instances:
    problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))

    coords = torch.tensor(problem['node_coord']).float()
    coords_norm = normalize_coord(coords)
    demand = torch.tensor(problem['demand'][1:]).float()
    capacity = problem['capacity']
    n = coords.shape[0]

    # Prepare the tensordict
    batch_size = 2
    td = env.reset(batch_size=(batch_size,)).to(device)
    td['locs'] = repeat(coords_norm, 'n d -> b n d', b=batch_size, d=2)
    td['demand'] = repeat(demand, 'n -> b n', b=batch_size) / capacity
    td['visited'] = torch.zeros((batch_size, 1, n), dtype=torch.uint8)
    action_mask = torch.ones(batch_size, n, dtype=torch.bool)
    action_mask[:, 0] = False
    td['action_mask']  = action_mask
    
    # Augmentation
    td = augmentation(td)

    # Get the solution from the policy
    with torch.no_grad():
        out = policy(
            td.clone(), decode_type='greedy', num_starts=0, return_actions=True
        )

    # Calculate the cost on the original scale
    coords_repeat = repeat(coords, 'n d -> b n d', b=batch_size, d=2)
    td['locs'] = batchify(coords_repeat, num_augment)
    reward = env.get_reward(td, out['actions'])
    reward = unbatchify(reward, num_augment)
    cost = ceil(-1 * torch.max(reward).item())

    # Load the optimal cost
    solution = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
    optimal_cost = solution['cost']

    # Calculate the gap and print
    gap = (cost - optimal_cost) / optimal_cost
    print(f'Problem: {instance:<15} Cost: {cost:<10} Optimal Cost: {optimal_cost:<10}\t Gap: {gap:.2%}')

Problem: A-n53-k7        Cost: 1123       Optimal Cost: 1010      	 Gap: 11.19%
Problem: A-n54-k7        Cost: 1305       Optimal Cost: 1167      	 Gap: 11.83%
Problem: A-n55-k9        Cost: 1199       Optimal Cost: 1073      	 Gap: 11.74%
Problem: A-n60-k9        Cost: 1534       Optimal Cost: 1354      	 Gap: 13.29%
Problem: A-n61-k9        Cost: 1187       Optimal Cost: 1034      	 Gap: 14.80%
Problem: A-n62-k8        Cost: 1474       Optimal Cost: 1288      	 Gap: 14.44%
Problem: A-n63-k9        Cost: 1820       Optimal Cost: 1616      	 Gap: 12.62%
Problem: A-n63-k10       Cost: 1505       Optimal Cost: 1314      	 Gap: 14.54%
Problem: A-n64-k9        Cost: 1582       Optimal Cost: 1401      	 Gap: 12.92%
Problem: A-n65-k9        Cost: 1332       Optimal Cost: 1174      	 Gap: 13.46%
Problem: A-n69-k9        Cost: 1305       Optimal Cost: 1159      	 Gap: 12.60%
Problem: A-n80-k10       Cost: 2044       Optimal Cost: 1763      	 Gap: 15.94%
Problem: B-n51-k7        Cost: 1073     

## Test the Sampling

In [21]:
# Parameters for sampling
num_samples = 100
softmax_temp = 0.05

for instance in instances:
    problem = vrplib.read_instance(os.path.join(path_to_save, instance+'.vrp'))

    coords = torch.tensor(problem['node_coord']).float()
    coords_norm = normalize_coord(coords)
    demand = torch.tensor(problem['demand'][1:]).float()
    capacity = problem['capacity']
    n = coords.shape[0]

    # Prepare the tensordict
    batch_size = 2
    td = env.reset(batch_size=(batch_size,)).to(device)
    td['locs'] = repeat(coords_norm, 'n d -> b n d', b=batch_size, d=2)
    td['demand'] = repeat(demand, 'n -> b n', b=batch_size) / capacity
    td['visited'] = torch.zeros((batch_size, 1, n), dtype=torch.uint8)
    action_mask = torch.ones(batch_size, n, dtype=torch.bool)
    action_mask[:, 0] = False
    td['action_mask']  = action_mask
    
    # Sampling
    td = batchify(td, num_samples)

    # Get the solution from the policy
    with torch.no_grad():
        out = policy(
            td.clone(), decode_type='sampling', num_starts=0, return_actions=True
        )

    # Calculate the cost on the original scale
    coords_repeat = repeat(coords, 'n d -> b n d', b=batch_size, d=2)
    td['locs'] = batchify(coords_repeat, num_samples)
    reward = env.get_reward(td, out['actions'])
    reward = unbatchify(reward, num_samples)
    cost = ceil(-1 * torch.max(reward).item())

    # Load the optimal cost
    solution = vrplib.read_solution(os.path.join(path_to_save, instance+'.sol'))
    optimal_cost = solution['cost']

    # Calculate the gap and print
    gap = (cost - optimal_cost) / optimal_cost
    print(f'Problem: {instance:<15} Cost: {cost:<10} Optimal Cost: {optimal_cost:<10}\t Gap: {gap:.2%}')

Problem: A-n53-k7        Cost: 1191       Optimal Cost: 1010      	 Gap: 17.92%
Problem: A-n54-k7        Cost: 1328       Optimal Cost: 1167      	 Gap: 13.80%
Problem: A-n55-k9        Cost: 1286       Optimal Cost: 1073      	 Gap: 19.85%
Problem: A-n60-k9        Cost: 1631       Optimal Cost: 1354      	 Gap: 20.46%
Problem: A-n61-k9        Cost: 1230       Optimal Cost: 1034      	 Gap: 18.96%
Problem: A-n62-k8        Cost: 1505       Optimal Cost: 1288      	 Gap: 16.85%
Problem: A-n63-k9        Cost: 1840       Optimal Cost: 1616      	 Gap: 13.86%
Problem: A-n63-k10       Cost: 1590       Optimal Cost: 1314      	 Gap: 21.00%
Problem: A-n64-k9        Cost: 1643       Optimal Cost: 1401      	 Gap: 17.27%
Problem: A-n65-k9        Cost: 1381       Optimal Cost: 1174      	 Gap: 17.63%
Problem: A-n69-k9        Cost: 1451       Optimal Cost: 1159      	 Gap: 25.19%
Problem: A-n80-k10       Cost: 2170       Optimal Cost: 1763      	 Gap: 23.09%
Problem: B-n51-k7        Cost: 1187     